**Author:** Prof. AJ Smit

Department of Biodiversity and Conservation Biology

University of the Western Cape

# Topic 13: Cluster analysis

In this example we will apply two types of cluster analyses, viz. **K-means clustering** and **hierarchical clustering**. Whereas ordination attempts to display the presence and influence of gradients, clustering tries to place our samples into a certain number of discrete units or clusters. We have seen that the WHO/SDG data seem to form neat groupings of countries within their respective parent locations. Let's explore this dataset with cluster analysis.

Additional examples of clustering to study are:

1. Numerical Ecology in R, pp. 53-62. Later pages in the Cluster chapter go deeper into clustering and you should read over it for a broad overview. For the purpose of this module, we will focus on 4.3 Hierarchical Clustering and 4.4 Agglomerative Clustering.
2. A [Kaggle challenge](https://www.kaggle.com/rohan0301/unsupervised-learning-on-country-data) with examples of both Hierarchical Clustering and K-means Clustering.

## Set-up the analysis environment

In [1]:
library(tidyverse) 
library(GGally)
library(cluster)
library(dendextend)
library(ggcorrplot)
library(factoextra)
library(gridExtra)
library(vegan)

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


---------------------
Welcome to dendextend version 1.15.1
Type citation('dendextend') for how to cite the package.

Type browseVignettes(package = 'dendextend') for the package vignette.
The github page is: https://github.com/talgalili/dendextend/

Suggestions and bug-reports can be submitted at: https://github.com/talgalili/dendextend/issues
Or contact: <tal.galili@gmail.com>

	To 

## Load the SDG data

I load the combined dataset that already had their missing values imputed (as per the [PCA](https://github.com/ajsmit/Quantitative_Ecology/blob/main/jupyter_lab/Topic_8-PCA-SDG-example.ipynb) example).

In [5]:
SDGs <- read_csv("/Users/ajsmit/Dropbox/R/workshops/Quantitative_Ecology/exercises/WHO/SDG_complete.csv")
head(SDGs)


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  ParentLocation = col_character(),
  Location = col_character()
)
ℹ Use `spec()` for the full column specifications.




ParentLocation,Location,other_1,other_2,SDG1.a,SDG16.1,SDG3.1_1,SDG3.2_1,SDG3.2_2,SDG3.2_3,⋯,SDG3.b_4,SDG3.c_1,SDG3.c_2,SDG3.c_3,SDG3.c_4,SDG3.d_1,SDG3.7,SDG3.a,SDG3.1_2,SDG3.b_5
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Eastern Mediterranean,Afghanistan,61.65,15.59,2.14,9.02,673,134.802054,230.59499,175.34049,⋯,62.00000,2.780000,1.480000,0.034000,0.4700000,42.00000,97.20786,14.35881,66.57461,68.86648
Europe,Albania,77.84,21.13,9.62,3.78,16,7.552973,11.31237,10.04785,⋯,98.00000,12.160000,36.500000,5.163544,6.4885512,82.02704,17.30000,29.70000,94.66754,56.50277
Africa,Algeria,76.54,21.81,10.73,1.66,113,37.999310,61.62882,53.23844,⋯,61.00000,18.330000,22.430000,4.262742,5.6060842,73.00000,43.97764,19.10000,87.91192,65.47076
Africa,Angola,61.72,16.71,5.43,9.82,246,124.853365,341.33898,228.84115,⋯,55.00000,-5.914274,-9.936513,-1.082242,-0.7075023,55.48743,111.43513,13.59252,61.05841,68.24041
Americas,Antigua and Barbuda,76.14,20.43,11.61,2.42,43,5.940594,10.89109,9.90099,⋯,85.61069,27.541210,44.870000,5.042694,6.3336920,81.00000,31.10000,23.60171,94.20909,55.99846
Americas,Argentina,76.17,20.98,13.47,6.23,40,11.006454,18.74016,16.69441,⋯,82.00000,40.010000,25.820000,5.358117,6.6479615,76.00000,25.93560,23.60000,98.40000,56.00000


The parent locations:

In [7]:
unique(SDGs$ParentLocation)
length(unique(SDGs$ParentLocation))

[1] "Eastern Mediterranean" "Europe"                "Africa"               
[4] "Americas"              "Western Pacific"       "South-East Asia"

[1] 6

The number of countries:

In [8]:
length(SDGs$Location)

[1] 176